In [ ]:
# Input any OneDrive URL 
onedrive_url = 'Paste here the link you copied from OneDrive'

import numpy as np
import pandas as pd
import requests, zipfile, io,base64

def create_onedrive_directdownload (onedrive_link):
    data_bytes64 = base64.b64encode(bytes(onedrive_link, 'utf-8'))
    data_bytes64_String = data_bytes64.decode('utf-8').replace('/','_').replace('+','-').rstrip("=")
    resultUrl = f"https://api.onedrive.com/v1.0/shares/u!{data_bytes64_String}/root/content"
    return resultUrl


# Generate Direct Download URL from above Script
direct_download_url = create_onedrive_directdownload(onedrive_url)


r = requests.get(direct_download_url)
z = zipfile.ZipFile(io.BytesIO(r.content))
df = pd.read_json(z.extract('Takeout/My Activity/Search/My Activity.json')).loc[:,['title','titleUrl','time']]

# You might want to change to your Time Zone
df['time'] = pd.to_datetime(df.time,infer_datetime_format=True,utc=True).dt.tz_convert('Europe/Copenhagen')
# Create a date column
df.loc[:,'date'] = pd.to_datetime(df['time']).dt.date

# Create a column for Visited and Searched for 
df = df.assign(action= lambda x: np.select(
    [x.title.str.startswith('Visited'),
     x.title.str.startswith('Searched for')],
    ['Visited','Searched for']))#.query("action=='0'")

# Extract only the text without (Visited & Searched for)
df = df.assign(text= lambda x: np.select(
    [x.title.str.startswith('Visited'),
     x.title.str.startswith('Searched for')],
    [x.title.str[8:],x.title.str[13:]]))

# Get length of text field
df = df.assign(text_len = lambda x: x.text.str.len())    

# Get first word form each Searched for row
df['first_word'] = np.where(df.action == 'Searched for',df.text.str.partition(' ')[0],None)